# Debug helpers

This notebook contains a bunch of scripts that can help debug some issues.

In [1]:
%%bash
# Ensure the required Python 3 dependencies are installed.
python3 -m pip install kafka-python

## Show the last 10 messages on a Kafka topic

In [34]:
import json
from datetime import datetime

from kafka import KafkaConsumer, TopicPartition

client = "localhost:9092"
consumer = KafkaConsumer(bootstrap_servers=['localhost:9092'],
                         enable_auto_commit=False,
                         auto_offset_reset='latest')

#
#  CHANGE THIS TO THE TOPIC YOU WANT TO DEBUG
#
topic = 'ingest-cleaned-oef2'


tp = TopicPartition(topic,0)
consumer.assign([tp])

consumer.seek_to_end(tp)
lastOffset = consumer.position(tp)
print(f"Last offset: {lastOffset}")

tailOffset = lastOffset - 10
if tailOffset < 0:
    print("Topic has no messages!")
else :
    consumer.seek(tp, tailOffset)    

    for message in consumer:
        print(f"RAW MESSAGE VALUE: {message.value}")
        try:
            m_obj = json.loads(message.value)
            print("RECODED JSON:", json.dumps(m_obj, indent=4))
        except:
            print("FAILED to decode message")
        ts = message.timestamp
        ts = datetime.fromtimestamp(ts/1000.0)
        print(f"TIMESTAMP: {ts}")
        print()
        if message.offset >= lastOffset - 1:
                break

Last offset: 603
RAW MESSAGE VALUE: b'{"date":"2016-01-06","hour":5,"window":{"start":"2016-01-06T05:00:00.000Z","end":"2016-01-06T06:00:00.000Z"},"numCalls":5,"historicalDate":"2016-01-05","historicalHour":5,"historicalNumCalls":12,"hasMoreCalls":false}'
RECODED JSON: {
    "date": "2016-01-06",
    "hour": 5,
    "window": {
        "start": "2016-01-06T05:00:00.000Z",
        "end": "2016-01-06T06:00:00.000Z"
    },
    "numCalls": 5,
    "historicalDate": "2016-01-05",
    "historicalHour": 5,
    "historicalNumCalls": 12,
    "hasMoreCalls": false
}
TIMESTAMP: 2020-12-03 15:48:07.309000

RAW MESSAGE VALUE: b'{"date":"2016-01-06","hour":6,"window":{"start":"2016-01-06T06:00:00.000Z","end":"2016-01-06T07:00:00.000Z"},"numCalls":10,"historicalDate":"2016-01-05","historicalHour":6,"historicalNumCalls":14,"hasMoreCalls":false}'
RECODED JSON: {
    "date": "2016-01-06",
    "hour": 6,
    "window": {
        "start": "2016-01-06T06:00:00.000Z",
        "end": "2016-01-06T07:00:00.000Z"


## Follow a Kafka topic

This script prints the content of messages of a Kafka topic when they get produced.

In [6]:
import json
from datetime import datetime
from IPython.display import clear_output

from kafka import KafkaConsumer, TopicPartition

client = "localhost:9092"
consumer = KafkaConsumer(bootstrap_servers=['localhost:9092'],
                         enable_auto_commit=False,
                         auto_offset_reset='latest')

#
#  CHANGE THIS TO THE TOPIC YOU WANT TO DEBUG
#
topic = 'ingest-cleaned-oef2'

consumer.subscribe(topic)

i = 0
loper=0
print(f"START LISTENING ON '{topic}'")
for message in consumer:
    loper = loper+1
    if i > 5:
        clear_output()
        i = 0
    i = i+1
    print(f"RAW MESSAGE VALUE: {message.value}")
    try:
        m_obj = json.loads(message.value)
        print("DECODED MESSAGE VALUE:", m_obj)
    except:
        print("FAILED to decode message")
    ts = message.timestamp
    ts = datetime.fromtimestamp(ts/1000.0)
    print(f"TIMESTAMP: {ts}")
    print()
print("Number of entries on kafka topic:", loper)

RAW MESSAGE VALUE: b'{"date":"2015-12-13","hour":6,"window":{"start":"2015-12-13T06:00:00.000Z","end":"2015-12-13T07:00:00.000Z"},"numCalls":8,"historicalDate":"2015-12-12","historicalHour":6,"historicalNumCalls":3,"hasMoreCalls":true}'
DECODED MESSAGE VALUE: {'date': '2015-12-13', 'hour': 6, 'window': {'start': '2015-12-13T06:00:00.000Z', 'end': '2015-12-13T07:00:00.000Z'}, 'numCalls': 8, 'historicalDate': '2015-12-12', 'historicalHour': 6, 'historicalNumCalls': 3, 'hasMoreCalls': True}
TIMESTAMP: 2020-12-03 14:02:25.913000

RAW MESSAGE VALUE: b'{"date":"2015-12-13","hour":7,"window":{"start":"2015-12-13T07:00:00.000Z","end":"2015-12-13T08:00:00.000Z"},"numCalls":13,"historicalDate":"2015-12-12","historicalHour":7,"historicalNumCalls":14,"hasMoreCalls":false}'
DECODED MESSAGE VALUE: {'date': '2015-12-13', 'hour': 7, 'window': {'start': '2015-12-13T07:00:00.000Z', 'end': '2015-12-13T08:00:00.000Z'}, 'numCalls': 13, 'historicalDate': '2015-12-12', 'historicalHour': 7, 'historicalNumCall

KeyboardInterrupt: 

## Delete a Kafka topic

In [25]:
from kafka.admin import KafkaAdminClient, NewTopic
from kafka.errors import TopicAlreadyExistsError

#
#  CHANGE THIS TO THE TOPIC YOU WANT TO DELETE
#
topic = 'ingest-test'

admin_client = KafkaAdminClient(bootstrap_servers="localhost:9092")
admin_client.delete_topics([topic])

DeleteTopicsResponse_v0(topic_error_codes=[(topic='ingest-test', error_code=0)])